# Pretraining both network parts seperatly

In [ ]:
from src.dlc_practical_prologue import generate_pair_sets
from src.utils import load_class_data, load_target_data, load_all_data
from src.models import *
from src.trainer import Trainer
import torch
import matplotlib.pyplot as plt

In [ ]:
# Load data with class and larget than as targets
dl_train_all, dl_val_all, dl_test_all = load_all_data()

# Load data with class as target
dl_train_class, dl_val_class, dl_test_class = load_class_data()

In [ ]:
print("--- Pre-training tail model ---\n")

tail = TailLinear(label_encoded=True)
trainer = Trainer(nb_epochs=5)
trainer.fit(tail, dl_train_all, dl_val_all)

print("\n--- Pre-train classification model ---\n")

le_net = LeNet()
trainer = Trainer(nb_epochs=15)
trainer.fit(le_net, dl_train_class, dl_val_class)

print("\n--- Putting them together ---\n")

# Putting them together
tail.label_encoded = False
siamese = Siamese(auxiliary=le_net, target=tail, weight_aux=0.3)
# trainer = Trainer(nb_epochs=5)
# trainer.fit(siamese, dl_train_all, dl_val_all)

print("\n--- Testing model ---\n")
trainer.test(le_net, dl_test_class, return_acc=False)
trainer.test(siamese, dl_test_all, return_acc=False)


In [4]:
# Do the same, but 10 times
results = []
for i in range(10):
    print(f"Run {i+1}/10")

    # Load data with class and larget than as targets
    dl_train_all, dl_val_all, dl_test_all = load_all_data()

    # Load data with class as target
    dl_train_class, dl_val_class, dl_test_class = load_class_data()

    # Training tail model
    tail = TailLinear(label_encoded=True)
    trainer = Trainer(nb_epochs=5)
    trainer.fit(tail, dl_train_all, dl_val_all, verbose=False)

    # Training leNet
    le_net = LeNet()
    trainer = Trainer(nb_epochs=15)
    trainer.fit(le_net, dl_train_class, dl_val_class, verbose=False)

    # Putting both together
    tail.label_encoded = False
    siamese = Siamese(le_net, tail)

    # Testing
    acc = trainer.test(siamese, dl_test_all, test_verbose=True, return_acc=True)
    results.extend([acc])

Run 1/10
# Epoch 1/15:	 loss=1.55	 loss_val=0.77	 acc_val=73.32
# Epoch 2/15:	 loss=0.5	 loss_val=0.38	 acc_val=87.98
# Epoch 3/15:	 loss=0.28	 loss_val=0.26	 acc_val=91.83
# Epoch 4/15:	 loss=0.2	 loss_val=0.26	 acc_val=92.07
# Epoch 5/15:	 loss=0.17	 loss_val=0.25	 acc_val=91.11
# Epoch 6/15:	 loss=0.14	 loss_val=0.21	 acc_val=93.75
# Epoch 7/15:	 loss=0.1	 loss_val=0.19	 acc_val=94.95
# Epoch 8/15:	 loss=0.07	 loss_val=0.17	 acc_val=95.43
# Epoch 9/15:	 loss=0.06	 loss_val=0.19	 acc_val=94.47
# Epoch 10/15:	 loss=0.05	 loss_val=0.19	 acc_val=94.47
# Epoch 11/15:	 loss=0.04	 loss_val=0.17	 acc_val=95.91
# Epoch 12/15:	 loss=0.03	 loss_val=0.2	 acc_val=95.19
# Epoch 13/15:	 loss=0.03	 loss_val=0.17	 acc_val=95.19
# Epoch 14/15:	 loss=0.03	 loss_val=0.2	 acc_val=95.43
# Epoch 15/15:	 loss=0.02	 loss_val=0.19	 acc_val=94.71
loss_test=0.61	 acc_test=76.17
Run 2/10
# Epoch 1/15:	 loss=1.49	 loss_val=0.61	 acc_val=81.01
# Epoch 2/15:	 loss=0.44	 loss_val=0.33	 acc_val=91.59


In [ ]:
# Print results
results = torch.Tensor(results)
print("Average accuracy:   {:.2f}".format(results.mean()))
print("Standard deviation: {:.2f}".format(results.std()))